In [1]:
import pandas as pd
import os
import glob

# --- Step 1: Load CSV files ---
csv_files = glob.glob("../data/*.csv")  # Adjust path or use absolute paths

In [2]:
# Read and concatenate
df_list = [
    pd.read_csv(file, on_bad_lines="warn").drop(
        columns=["KccAns", "Year", "Month", "BlockName"]
    )
    for file in csv_files
]

df = pd.concat(df_list, ignore_index=True)

/var/folders/37/hnn7hl3j01z6rnvld0st0lg40000gn/T/ipykernel_95133/470575609.py:3: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv(file, on_bad_lines="warn").drop(
/var/folders/37/hnn7hl3j01z6rnvld0st0lg40000gn/T/ipykernel_95133/470575609.py:3: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv(file, on_bad_lines="warn").drop(
/var/folders/37/hnn7hl3j01z6rnvld0st0lg40000gn/T/ipykernel_95133/470575609.py:3: ParserWarning: Skipping line 16251: expected 13 fields, saw 17

  pd.read_csv(file, on_bad_lines="warn").drop(


In [ ]:
df["CreatedOn"] = pd.to_datetime(df["CreatedOn"], format="mixed", errors="coerce")

In [9]:
df["DistrictName"] = df["DistrictName"].astype(str)
df["DistrictName"] = df["DistrictName"].str.strip()
df["DistrictName"] = df["DistrictName"].replace("", "Unknown")
df["Season"] = df["Season"].fillna("Unknown")

In [10]:
df["Sector"] = df["Sector"].astype(str)
df["Sector"] = df["Sector"].str.strip()  # Remove leading/trailing whitespace
df["Sector"] = df["Sector"].replace(
    ["9999", "256", "825"], "Unknown"
)  # Replace empty strings with 'Unknown'

In [11]:
df["Category"] = df["Category"].astype(str)
df["Category"] = df["Category"].str.strip()  # Remove leading/trailing whitespace
df["Category"] = df["Category"].replace(
    ["0", 0, "418", "417", "-1", "468"], "Unknown"
)  # Replace empty strings with 'Unknown'
df["Category"] = df["Category"].fillna("Unknown")

In [12]:
df["Crop"] = df["Crop"].astype(str)
df["Crop"] = df["Crop"].fillna("Unknown")
df["Crop"] = df["Crop"].str.strip()  # Remove leading/trailing whitespace

In [13]:
df["QueryType"] = df["QueryType"].astype(str)
df["QueryType"] = df["QueryType"].str.strip()  # Remove leading/trailing whitespace
df["QueryType"] = df["QueryType"].fillna("Unknown")

In [14]:
df["QueryText"] = df["QueryText"].astype(str)
df["QueryText"] = df["QueryText"].str.strip()  # Remove leading/trailing whitespace
df["QueryText"] = df["QueryText"].dropna()

In [15]:
# --- Step 2: Memory before optimization ---
print("Memory usage before compression:")
print(df.memory_usage(deep=True).sum() / (1024**2), "MB")

for col in df.select_dtypes(include="object").columns:
    num_unique = df[col].nunique()
    num_total = len(df[col])
    if num_unique / num_total < 0.02:  # Low cardinality
        df[col] = df[col].astype("category")

Memory usage before compression:
25291.230038642883 MB


In [16]:
# --- Step 3: Save compressed DataFrame ---
df.to_parquet("../data/compressed_df.parquet", engine="pyarrow", compression="snappy")

In [17]:
# --- Step 4: Memory after optimization ---
print("Memory usage after compression:")
print(df.memory_usage(deep=True).sum() / (1024**2), "MB")

# --- Step 5: Get total size of the original CSVs ---
total_csv_size = sum(os.path.getsize(file) for file in csv_files)
print("Total size of CSV files on disk:")
print(total_csv_size / (1024**2), "MB")

Memory usage after compression:
5087.791198730469 MB
Total size of CSV files on disk:
15907.595109939575 MB


In [ ]:
# Uncomment the following lines to read the parquet file

# import pandas as pd
# df = pd.read_parquet('../data/compressed_df.parquet', engine='pyarrow')